# Character based RNN language model trained on 'The Complete Works of William Shakespeare'
(c) Deniz Yuret, 2018. Based on http://karpathy.github.io/2015/05/21/rnn-effectiveness. **TODO:** Add julia/base example.

* Objectives: Learn to define and train a character based language model and generate text from it. Minibatch blocks of text. Keep a persistent RNN state between updates.
* Prerequisites: [RNN basics](06.rnn.ipynb), minibatch, param, param0, RNN, dropout, train!, Adam, nll

In [4]:
using Pkg
for p in ("Knet","ProgressMeter")
    haskey(Pkg.installed(),p) || Pkg.add(p)
end

In [5]:
RNNTYPE = :lstm
BATCHSIZE = 256
SEQLENGTH = 100
INPUTSIZE = 168
VOCABSIZE = 84
HIDDENSIZE = 334
NUMLAYERS = 1
DROPOUT = 0.0
LR=0.001
BETA_1=0.9
BETA_2=0.999
EPS=1e-08
EPOCHS = 30
ENV["COLUMNS"]=92;

## Load and minibatch data

In [6]:
# Load 'The Complete Works of William Shakespeare'
using Knet: Knet
include(Knet.dir("data","gutenberg.jl"))
trn,tst,chars = shakespeare()
map(summary,(trn,tst,chars))

("4934845-element Array{UInt8,1}", "526731-element Array{UInt8,1}", "84-element Array{Char,1}")

In [7]:
# Print a sample
println(string(chars[trn[1020:1210]]...)) 


    Cheated of feature by dissembling nature,
    Deform'd, unfinish'd, sent before my time
    Into this breathing world scarce half made up,
    And that so lamely and unfashionable
 


In [8]:
# Minibatch data
using Knet: minibatch
function mb(a)
    N = length(a) ÷ BATCHSIZE
    x = reshape(a[1:N*BATCHSIZE],N,BATCHSIZE)' # reshape full data to (B,N) with contiguous rows
    minibatch(x[:,1:N-1], x[:,2:N], SEQLENGTH) # split into (B,T) blocks 
end
dtrn,dtst = mb.((trn,tst))
length.((dtrn,dtst))

(192, 20)

In [9]:
summary.(first(dtrn))  # each x and y have dimensions (BATCHSIZE,SEQLENGTH)

("256×100 Array{UInt8,2}", "256×100 Array{UInt8,2}")

## Define and initialize model

In [10]:
using Knet: param, param0, RNN, dropout

In [11]:
struct CharLM; input; rnn; output; end

CharLM(vocab::Int,input::Int,hidden::Int; o...) = 
    CharLM(Embed(vocab,input), RNN(input,hidden; o...), Linear(hidden,vocab))

function (c::CharLM)(x; pdrop=0, hidden=nothing)
    x = c.input(x)                # (B,T)->(X,B,T)
    x = dropout(x, pdrop)
    x = c.rnn(x, hidden=hidden)   # (H,B,T)
    x = dropout(x, pdrop)
    x = reshape(x, size(x,1), :)  # (H,B*T)
    return c.output(x)            # (V,B*T)
end

In [12]:
struct Embed; w; end

Embed(vocab::Int,embed::Int)=Embed(param(embed,vocab))

(e::Embed)(x) = e.w[:,x]

In [13]:
struct Linear; w; b; end

Linear(input::Int, output::Int)=Linear(param(output,input), param0(output))

(l::Linear)(x) = l.w * x .+ l.b

In [19]:
# For running experiments
using Knet: train!, Adam, AutoGrad; import ProgressMeter
function trainresults(file,model,chars)
    if (print("Train from scratch? ");readline()[1]=='y')
        updates = 0; prog = ProgressMeter.Progress(EPOCHS * length(dtrn))
        callback(J)=(ProgressMeter.update!(prog, updates); (updates += 1) <= prog.n)
        opt = Adam(lr=LR, beta1=BETA_1, beta2=BETA_2, eps=EPS)
        train!(model, dtrn; callback=callback, optimizer=opt, pdrop=DROPOUT, hidden=[])
        Knet.gc(); Knet.save(file,"model",model,"chars",chars)
    else
        isfile(file) || download("http://people.csail.mit.edu/deniz/models/tutorial/$file",file)
        model,chars = Knet.load(file,"model","chars")
    end
    return model,chars
end

trainresults (generic function with 2 methods)

In [20]:
clm,chars = trainresults("shakespeare.jld2", 
    CharLM(VOCABSIZE, INPUTSIZE, HIDDENSIZE; rnnType=RNNTYPE, numLayers=NUMLAYERS, dropout=DROPOUT), chars);

Train from scratch? stdin> y


Progress: 100%|█████████████████████████████████████████████████████| Time: 0:02:03


In [21]:
using Knet: nll
exp(nll(clm,dtst))  # Perplexity

4.328634f0

In [22]:
# Sample from trained model

function generate(model,chars,n)
    function sample(y)
        p = Array(exp.(y)); r = rand()*sum(p)
        for j=1:length(p); (r -= p[j]) < 0 && return j; end
    end
    x = 1
    h = []
    for i=1:n
        y = model([x], hidden=h)
        x = sample(y)
        print(chars[x])
    end
    println()
end;

In [23]:
generate(clm,chars,1000)

FORT. This man cry, making figures a court;
  nound for lecdles your folly would be,
  Ley thus base at us wall; and be, give ho,
    I will not lose a drunken. The wreck of grace,
  A parled in two beauty to the steed
     Apon my master, will, would know hold your gapey;
            But heaven with our father, Edward shall say of York, and amised Taura
      Only thou dost to the ancient deek shed?
  Gon. But to keep the elenatures,  
     I am some dine, i' faith.                              Exit
    Here the Lovery men stay'd, farewell. What!
    Let this address of Italizen
    Break the ruin, that he swaggerers of our trees,
    My want weary appromise a sister,
    Whose strife they have free to lose his meet.
                       Exeunt LORDS, and EDWARD, after parts
    Have I do think?
    Cliff him, my breath of your thing towards Camillo,
    Then taking the curse, by these sword are implaints
    That what I did give an end idle breath,
    Your peac
